In [2]:
########## SET-UP ##########

# necessary libraries
import os
import json
import requests
import time
import datetime

# INSERT YOUR API KEY HERE
KEY = 'b00c992b-6fc1-4795-abcc-7db85c1b72fa'


In [113]:
jaemin_recent_games = get_recent_games('na', 61281860)
jaemin_teams = get_teams_by_summonerid('na', 61281860)
#print jaemin_recent_games.text
#print jaemin_teams.text

In [2]:
########## Champion ID to Champion Name converter ##########

# GET the static data on champion keys and names
static_champ_data = requests.get("https://global.api.pvp.net/api/lol/static-data/na/v1.2/champion?locale=en_US&api_key=b00c992b-6fc1-4795-abcc-7db85c1b72fa").json()
champs = static_champ_data['data']
champ_names = static_champ_data['data'].keys()

# initializing index and empty dictionaries
i = 0
list_champs = []
list_champ_ids = []

# filling in the lists
for i in range(len(champ_names)):
    this_champ = champs[champ_names[i]]
    list_champs.append(this_champ['key'])
    list_champ_ids.append(this_champ['id'])
    
# champion ID and name dictionary    
champ_dict = dict(zip(list_champ_ids, list_champs))

# Wukong's name is not correct (MonkeyKing)
champ_dict[62] = 'Wukong'

# function that converts a champion ID to champion name
def champion_name(champion_id):
    if champion_id in champ_dict:
        return champ_dict[champion_id]
    else:
        return 'no_name'


In [3]:
# MATCH-v2.2

REGION_ENDPOINT = "https://{0}.api.pvp.net/api/lol/{0}/"

def get_match(REGION, matchId, includeTimeline):
    """
    Retrieve match by match ID.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.2/match/{1}?"
         "api_key={2}&includeTimeline={3}").
        format(REGION, matchId, KEY, includeTimeline))

In [7]:
test_response = get_match('na', 2026610783, True).json()
#test_response = get_match('na', 2027658798, True)
#print test_response['highestAchievedSeasonTier']
#print test_response['season']
#print test_response['participants'][1]['highestAchievedSeasonTier']
print test_response['participants'][0]['stats
#print test_response.json()
#print test_response['queueType']

{u'stats': {u'item1': 3069, u'totalPlayerScore': 0, u'unrealKills': 0, u'objectivePlayerScore': 0, u'largestCriticalStrike': 0, u'totalDamageDealt': 14753, u'magicDamageDealtToChampions': 4025, u'largestMultiKill': 2, u'largestKillingSpree': 2, u'quadraKills': 0, u'magicDamageTaken': 2804, u'towerKills': 1, u'totalTimeCrowdControlDealt': 163, u'neutralMinionsKilledEnemyJungle': 0, u'firstTowerAssist': False, u'neutralMinionsKilledTeamJungle': 0, u'wardsKilled': 6, u'physicalDamageDealt': 3795, u'item2': 3158, u'item3': 3105, u'item0': 2049, u'winner': True, u'item6': 3341, u'wardsPlaced': 16, u'item4': 2043, u'item5': 3067, u'minionsKilled': 20, u'tripleKills': 0, u'champLevel': 10, u'magicDamageDealt': 10958, u'kills': 2, u'doubleKills': 1, u'firstInhibitorKill': False, u'trueDamageTaken': 96, u'firstBloodAssist': False, u'firstBloodKill': False, u'assists': 8, u'deaths': 1, u'neutralMinionsKilled': 0, u'combatPlayerScore': 0, u'visionWardsBoughtInGame': 4, u'physicalDamageDealtToCham

In [9]:
# Constants
REGION = 'na'
STATUS_OK = 200
STATUS_RATE_LIMIT_EXCEEDED = 429
valid_rank_games = ['RANKED_SOLO_5x5',  # Ranked Solo 5v5 games
                    'RANKED_PREMADE_3x3',  # Ranked Premade 3v3 games
                    'RANKED_PREMADE_5x5',  # Ranked Premade 5v5 games
                    'RANKED_TEAM_3x3',  # Ranked Team 3v3 games
                    'RANKED_TEAM_5x5']  # Ranked Team 5v5 games

# Starting MatchID
STARTING_MATCH_ID = 2026937933

# Functions that check status of the data pulls
def isValid(match):
    if match.status_code != STATUS_OK:
        return False
    return True

def isClassicMatch(match):
    j = match.json()
    if j['mapId'] == 11 and j['matchMode'] == "CLASSIC" and j['matchType'] == 'MATCHED_GAME':
        return True
    else:
        return False

def isRankMatch(match):
    j = match.json()
    if j['queueType'] in valid_rank_games:
        return True
    else:
        return False
                    
def rateLimitExceeded(match):
    if int(match.status_code) == STATUS_RATE_LIMIT_EXCEEDED:
        return True
    return False

# index for the for-loop
i = 0
# counting the number of data retrieved
n = 0
# counting how many times we sleep in case we hit a dead-end
sleep_counter = 0

for i in range(100000):
    m = get_match(REGION, i + STARTING_MATCH_ID, True)
    
    if not rateLimitExceeded(m):
        if isValid(m) and isClassicMatch(m) and isRankMatch(m):
            f = open("dump/" + str(i + STARTING_MATCH_ID)+".json", 'w+')
            f.write(m.text.encode('utf-8'))
            n += 1
            print 'n: %d, ts: %s, gamdID: %d, date: %s, season: %s, tier: %s, loop_index: %d' % (n, datetime.datetime.fromtimestamp(time.time()).strftime('%m-%d %H:%M:%S'), 
                                                                                                 i + STARTING_MATCH_ID, 
                                                                                                 ('%s' % (time.strftime('%m-%d', time.localtime(m.json()['matchCreation']/1000)))),
                                                                                                 m.json()['season'], m.json()['participants'][1]['highestAchievedSeasonTier'], i)
            f.close()
            i += 1
            # reset sleep_counter if we are get data again
            sleep_counter = 0
    else:
        sleep_counter += 1
        print("API rate limit reached - waiting 10 seconds, sleep count: %d, loop_index: %d") % (sleep_counter, i)
        time.sleep(10)
        i += 1

API rate limit reached - waiting 10 seconds, sleep count: 1
API rate limit reached - waiting 10 seconds, sleep count: 2
API rate limit reached - waiting 10 seconds, sleep count: 3
n: 1, ts: 12-05 12:51:17, gamdID: 2026937814, date: 12-03, season: PRESEASON2016, tier: SILVER, loop_index: 24
API rate limit reached - waiting 10 seconds, sleep count: 1
n: 2, ts: 12-05 12:51:29, gamdID: 2026937818, date: 12-03, season: PRESEASON2016, tier: SILVER, loop_index: 28
n: 3, ts: 12-05 12:51:31, gamdID: 2026937822, date: 12-03, season: PRESEASON2016, tier: UNRANKED, loop_index: 32
API rate limit reached - waiting 10 seconds, sleep count: 1
API rate limit reached - waiting 10 seconds, sleep count: 2
API rate limit reached - waiting 10 seconds, sleep count: 3
API rate limit reached - waiting 10 seconds, sleep count: 4
API rate limit reached - waiting 10 seconds, sleep count: 5
API rate limit reached - waiting 10 seconds, sleep count: 6
API rate limit reached - waiting 10 seconds, sleep count: 7
n: 4,

KeyboardInterrupt: 